In [1]:
import os
import pinecone

In [2]:
from pinecone.grpc import PineconeGRPC as Pinecone 
from pinecone import ServerlessSpec

In [3]:
pc =Pinecone(api_key="")

In [4]:
spec = ServerlessSpec(cloud='aws',region='us-east-1')

In [5]:
index_name = 'keys'

In [6]:
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

In [7]:
dimension = 384
pc.create_index(
    name=index_name,
    dimension=dimension,
    metric = 'cosine',
    spec = spec
)

In [8]:
index = pc.Index(index_name)

In [9]:
from sentence_transformers import  SentenceTransformer
import pandas as pd

In [10]:
model = SentenceTransformer('all-MiniLM-L6-v2')

C:\Users\harsh\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
def embed_text(text):
    return model.encode(text)

In [55]:
df = pd.read_csv('RAG.csv',header=None)
columns = [i for i in range(2,15) ]
df = df.drop(df[columns] , axis = 1)
df = df.dropna()
df

,0,1
0,"According to Wikipedia , Jupiter is the fifth ...","source: wikipedia , name:jupiter , fifth plane..."
1,"According to Wikipedia , Mars is the fourth pl...","source: Wikipedia, name: Mars, fourth planet f..."
2,"According to Wikipedia , Greek (Modern Greek: ...","source: Wikipedia, name: Greek, language famil..."
3,"According toWikipedia , Virat Kohli (Hindi pro...","source: Wikipedia, name: Virat Kohli, born: 5 ..."
4,"According to Wikipedia , India, officially the...","source: Wikipedia, name: India, official name:..."
5,"According to Britannica, Mount Everest, mounta...","source: Britannica, name: Mount Everest, locat..."
6,"According to WWF , Rivers are unpredictable, a...","source: WWF, region: Amazon River Basin, chara..."
7,According to International Fund for Animal Wel...,"Name: Great White Shark, Largest predatory fis..."
8,"According to wikipedia, Interstellar is a 2014...","Source: Wikipedia , Name: Interstellar , Relea..."
9,Serbian-American engineer and physicist Nikola...,"source: Wikipedia , name: Nikola Tesla , born:..."


In [61]:
key_list = list()
for i in df[1]:
    key_list.append(i)

In [62]:
Full_text_list = list()
for i in df[1]:
    Full_text_list.append(i)

In [65]:
for i in range(len(key_list)):
    embedding = embed_text(key_list[i])
    index.upsert(vectors = [(f"key_{i}",embedding.tolist() , {"text":key_list[i]})])    

C:\Users\harsh\AppData\Roaming\Python\Python312\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [73]:
def search_DB(query):
    query_embedding = embed_text(query)
    result = index.query(
        vector=query_embedding.tolist(),
        top_k=3,
        include_metadata=True
    )
    return result['matches']

In [74]:
ques = input()
search_DB(ques)

 mass of jupiter


[{'id': 'key_0',
  'metadata': {'text': 'source: wikipedia , name:jupiter , fifth planet from '
                       'sun , largest planet in solar system , mass: 2.5times '
                       'of all other planets etc'},
  'score': 0.5779819,
  'sparse_values': {'indices': [], 'values': []},
  'values': []},
 {'id': 'key_1',
  'metadata': {'text': 'source: Wikipedia, name: Mars, fourth planet from the '
                       'Sun, nickname: the Red Planet, surface color: '
                       'orange-red (due to iron(III) oxide dust), brightness: '
                       'among the brightest objects in Earth’s sky, '
                       'classification: terrestrial planet, size: second '
                       'smallest in the Solar System, diameter: 6,779 km (4,212 '
                       'mi), Martian day (sol): 24.5 hours, Martian year: 1.88 '
                       'Earth years (687 Earth days), satellites: two (Phobos '
                       'and Deimos)'},
  'scor

In [69]:
import ollama

In [70]:
stream = ollama.chat(
  model='llama3.2:3b',
  messages=[{'role': 'user', 'content': 'Name all planets in solar system'}],
  stream=True
)
for chunk in stream:
    print(chunk['message']['content'],end='',flush=True)

Here are the 8 planets in our solar system, in order from the Sun:

1. Mercury
2. Venus
3. Earth
4. Mars
5. Jupiter
6. Saturn
7. Uranus
8. Neptune

In [75]:
def answer(qeury):
    context = search_DB(query)
    stream = ollama.chat(
    model='llama3.2:3b',
    messages=[{'role': 'user', 'content': f'Based on this content{context} , answer the {query}'}],
    stream=True
    )
    for chunk in stream:
        print(chunk['message']['content'],end='',flush=True)    

In [77]:
query = input()
answer(query)

 what is Joint Entrance Exam ?


The Joint Entrance Examination (JEE) is an engineering entrance assessment in India that serves as a gateway to admission into various engineering colleges in India. It consists of two main components: JEE-Main and JEE-Advanced.

Here's a brief overview:

1. **Purpose:** The primary purpose of the JEE is to assess the academic ability and aptitude of students seeking admission into Indian Institutes of Technology (IITs), National Institutes of Technology (NITs), and other engineering colleges in India.
2. **Components:**
	* **JEE-Main:** A paper-based examination that tests a student's understanding of mathematics, physics, and chemistry. It is designed to assess the basic mathematical skills required for undergraduate studies.
	* **JEE-Advanced:** An online examination that tests a student's problem-solving skills in mathematics, physics, and chemistry. It is designed to assess the advanced mathematical skills required for admission into IITs.
3. **Admission basis:** Admission to engi